In [1]:
import pandas as pd
from flat_processing_job.database.connection import get_engine_and_session

engine, session = get_engine_and_session()

AssertionError: No .env file found, have you copied .env.tmpl to .env?

In [2]:
# Load the data
data = pd.read_sql_table("partially_parsed_flats", engine)

In [ ]:
data.columns

In [4]:
data = data.drop_duplicates(subset=["description"])

In [5]:
data["price"] = data["price"].apply(lambda x: x.replace("€", "").replace(" ", "")).astype(int)

In [6]:
data["area"] = data["area"].apply(lambda x: x.replace("m²", "").replace(",", ".")).astype(float)

In [7]:
# disable max cols
pd.set_option("display.max_columns", None)

In [ ]:
data.head()

In [9]:
data[["room_count", "floor", "total_floors"]] = data[["room_count", "floor", "total_floors"]].astype(int)

In [ ]:
data.dtypes

In [ ]:
data.head()

In [12]:
data["energy_class"] = data["energy_class"].fillna("Unknown")
data[["features", "additional_rooms", "additional_equipment", "security"]] = data[["features", "additional_rooms", "additional_equipment", "security"]].fillna("None")


In [13]:
list_cols = ["features", "additional_rooms", "additional_equipment", "security"]
for col in list_cols:
    data[col] = data[col].apply(lambda x: x.split("\n"))


In [14]:
for col in list_cols:
    data[col] = data[col].apply(lambda x: [y.strip() for y in x])

In [16]:
features = {
    "separate_entrance": "Atskiras įėjimas",
    "high_ceilings": "Aukštos lubos",
    "attic_apartment": "Butas palėpėje",
    "multi_floor_apartment": "Butas per kelis aukštus",
    "address_registration": "Galima deklaruoti gyvenamąją vietą",
    "pet_friendly": "Galima su gyvūnais",
    "internet": "Internetas",
    "cable_tv": "Kabelinė televizija",
    "flexible_rent_for_ukrainians": "Lankstesnės nuomos sąlygos ukrainiečiams (dėl nuomos laikotarpio, kainos ar kt.). Daugiau informacijos – skelbime nurodytais kontaktais. / Більш гнучкі умови оренди для українців (за рахунок терміну оренди, цін тощо). Більше інформації за контактами, вказаними в оголошенні.",
    "new_electrical_wiring": "Nauja elektros instaliacija",
    "new_sewage_system": "Nauja kanalizacija",
    "separate_wc_and_bathroom": "Tualetas ir vonia atskirai",
    "gated_yard": "Uždaras kiemas",
    "kitchen_living_room": "Virtuvė sujungta su kambariu",
    "elevator": "Yra liftas",
    "help_for_ukrainians": "Допомога українцям",
}

additional_rooms = {
    "balcony": "Balkonas",
    "walk_in_closet": "Drabužinė",
    "sauna": "Pirtis",
    "basement": "Rūsys",
    "storage_room": "Sandėliukas",
    "terrace": "Terasa",
    "parking_space": "Vieta automobiliui",
    "attic": "Yra palėpė",
}


additional_equipment = {
    "shower_cabin": "Dušo kabina",
    "dishwasher": "Indaplovė",
    "air_conditioner": "Kondicionierius",
    "plastic_pipes": "Plastikiniai vamzdžiai",
    "ventilation_system": "Rekuperacinė sistema",
    "washing_machine": "Skalbimo mašina",
    "furnished": "Su baldais",
    "kitchen_set": "Virtuvės komplektas",
    "stove": "Viryklė",
    "bathtub": "Vonia",
    "refrigerator": "Šaldytuvas",
    "heated_floors": "Šildomos grindys",
    "fireplace": "Židinys",
}

security = {
    "security_guard": "Budintis sargas",
    "coded_staircase_lock": "Kodinė laiptinės spyna",
    "alarm_system": "Signalizacija",
    "cctv": "Vaizdo kameros",
    "reinforced_door": "Šarvuotos durys",
}


In [19]:
property_features = {
    "features": {
        "separate_entrance": "Atskiras įėjimas",
        "high_ceilings": "Aukštos lubos",
        "attic_apartment": "Butas palėpėje",
        "multi_floor_apartment": "Butas per kelis aukštus",
        "address_registration": "Galima deklaruoti gyvenamąją vietą",
        "pet_friendly": "Galima su gyvūnais",
        "internet": "Internetas",
        "cable_tv": "Kabelinė televizija",
        "flexible_rent_for_ukrainians": "Lankstesnės nuomos sąlygos ukrainiečiams (dėl nuomos laikotarpio, kainos ar kt.). Daugiau informacijos – skelbime nurodytais kontaktais. / Більш гнучкі умови оренди для українців (за рахунок терміну оренди, цін тощо). Більше інформації за контактами, вказаними в оголошенні.",
        "new_electrical_wiring": "Nauja elektros instaliacija",
        "new_sewage_system": "Nauja kanalizacija",
        "separate_wc_and_bathroom": "Tualetas ir vonia atskirai",
        "gated_yard": "Uždaras kiemas",
        "kitchen_living_room": "Virtuvė sujungta su kambariu",
        "elevator": "Yra liftas",
        "help_for_ukrainians": "Допомога українцям",
    },
    "additional_rooms": {
        "balcony": "Balkonas",
        "walk_in_closet": "Drabužinė",
        "sauna": "Pirtis",
        "basement": "Rūsys",
        "storage_room": "Sandėliukas",
        "terrace": "Terasa",
        "parking_space": "Vieta automobiliui",
        "attic": "Yra palėpė",
    },
    "additional_equipment": {
        "shower_cabin": "Dušo kabina",
        "dishwasher": "Indaplovė",
        "air_conditioner": "Kondicionierius",
        "plastic_pipes": "Plastikiniai vamzdžiai",
        "ventilation_system": "Rekuperacinė sistema",
        "washing_machine": "Skalbimo mašina",
        "furnished": "Su baldais",
        "kitchen_set": "Virtuvės komplektas",
        "stove": "Viryklė",
        "bathtub": "Vonia",
        "refrigerator": "Šaldytuvas",
        "heated_floors": "Šildomos grindys",
        "fireplace": "Židinys",
    },
    "security": {
        "security_guard": "Budintis sargas",
        "coded_staircase_lock": "Kodinė laiptinės spyna",
        "alarm_system": "Signalizacija",
        "cctv": "Vaizdo kameros",
        "reinforced_door": "Šarvuotos durys",
    },
}


for col, values in property_features.items():
    for feature, value in values.items():
        data[feature] = data[col].apply(lambda x: feature in x)  # noqa: B023

In [20]:
data[["latitude", "longitude"]] = data["coordinates"].apply(pd.Series).astype(float)

In [ ]:
data[["latitude", "longitude"]]

In [ ]:
list(property_features.keys())

In [23]:
data = data.drop(list(property_features.keys()), axis=1)
data = data.drop("coordinates", axis=1)
data = data.drop("images", axis=1)
data = data.drop(["nearest_kindergarten", "nearest_school", "nearest_store"], axis=1)


In [ ]:
data

In [28]:
data.isna().sum().to_clipboard()

In [ ]:
def extract_renovation_year(year_str: str) -> int | None:
	if "renovacija" not in year_str.lower():
		return None
	return year_str.split(",")[-1][:4]

def extract_construction_year(year_str: str) -> int | None:
	if "statyba" in year_str.lower():
		return int(year_str[:4])
	return int(year_str)

data["construction_year"] = data["year"].apply(extract_construction_year)
data["renovation_year"] = data["year"].apply(extract_renovation_year)

In [ ]:
data